In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ChurnEDA") \
    .getOrCreate()

spark

In [2]:
df = spark.read.csv("../data/Telco_customer_churn.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)
 |-- churn_rate: integer (nullable = true)
 |-- Churn Score: integer (nullable = t

In [3]:
df.groupBy("Churn").count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|   No| 5174|
|  Yes| 1869|
+-----+-----+



In [7]:
df.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn',
 'churn_rate',
 'Churn Score',
 'CLTV',
 'Churn Reason',
 'Count',
 'Country',
 'State',
 'City',
 'Zip Code',
 'Lat Long',
 'Latitude',
 'Longitude',
 'Age',
 'Under 30',
 'Married',
 'Referred a Friend',
 'Number of Referrals',
 'Offer',
 'Avg Monthly Long Distance Charges',
 'Avg Monthly GB Download',
 'Streaming Music',
 'Premium Tech Support',
 'Unlimited Data',
 'Total Refunds',
 'Total Extra Data Charges',
 'Total Long Distance Charges',
 'Total Revenue',
 'Satisfaction Score',
 'Customer Status',
 'Churn Score.1',
 'Churn Category']

In [9]:
for old_name in df.columns:
    new_name = old_name.strip().replace(" ", "_").replace(".", "_")
    if old_name != new_name:
        df = df.withColumnRenamed(old_name, new_name)
df.columns
df.describe().show()

+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+------------------+------------------+------------------+--------------------+-----+-------------+----------+------+------------------+--------------------+-----------------+------------------+-----------------+--------+-------+-----------------+-------------------+-------+---------------------------------+-----------------------+---------------+--------------------+--------------+------------------+------------------------+---------------------------+------------------+------------------+---------------+------------------+--------------+
|summary|customerID|gender|     SeniorCitizen|Partner|Dependents|            tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|Onli